In [34]:
import random
import sqlite3


from google.cloud import bigquery
from google.oauth2 import service_account

In [35]:
service_path = "/Users/krusty/Desktop/MSBA /GBQ Key/"
service_file = 'my-project-366122-3a6c8a47fb52.json'  
gbq_proj_id = 'my-project-366122' 
gbq_dataset_id = 'wedge'
private_key =service_path + service_file

In [36]:
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

## Create three tables: 
sales by date hour, sales by owner by year/month, sales by product description by year/month

## Query 1 : Sales by Date by Hour

In [37]:
#remember appendix requirements 
sdhtable = '''SELECT (EXTRACT(date FROM datetime)) AS Date,
    (EXTRACT(HOUR FROM datetime)) AS Hour,
    SUM(total) as Sales,
    COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
    SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `my-project-366122.wedge.transArchive*`
    
    WHERE card_no != 3 
    AND department != 0
    AND department != 15
    AND trans_status != 'M'
    AND trans_status != 'C'
    AND trans_status != 'J'
    AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
    GROUP BY Date, Hour
    ORDER BY Date, Hour'''

date_hour_trans = client.query(sdhtable,location="US")

headers = ["Date","Hour","Sales","Transactions","Items"]

In [38]:
with open("sales_date_hour.txt", 'w') as outfile:
    outfile.write(",".join(headers) + "\n")
    for row in date_hour_trans:
         outfile.write(",".join([str(item) for item in row])+ '\n')

In [42]:
db = sqlite3.connect("Wedge3.db")

In [40]:
cur = db.cursor()

In [41]:
input_file1 = "sales_date_hour.txt"

cur.execute('''DROP TABLE IF EXISTS sales_date_hour''') 

cur.execute('''CREATE TABLE sales_date_hour (
    Date TIMESTAMP,
    Hour TIMESTAMP,
    Sales FLOAT,
    Transactions INTEGER,
    Items INTEGER)''')

with open(input_file1, 'r', encoding = 'Latin-1') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        cur.execute('''
        INSERT INTO sales_date_hour (Date, Hour, Sales, Transactions, Items)
        VALUES (?,?,?,?,?)''', line) 
db.commit()

## Query 2 - Sales by Owner by Year/Month

In [13]:
#rinse repeat 
sdhtable2 = '''SELECT card_no,
    (EXTRACT(year FROM datetime)) AS Year,
    (EXTRACT(month FROM datetime)) AS Month,
    SUM(total) as Sales,
    COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
    SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `my-project-366122.wedge.transArchive*`
    
    WHERE department != 0
    AND department != 15
    AND trans_status != 'M'
    AND trans_status != 'C'
    AND trans_status != 'J'
    AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
    GROUP BY card_no, Year, Month
    ORDER BY card_no, Year, Month'''

owner_year_month = client.query(sdhtable2,location="US")

headers = ["Owner_ID","Year","Month","Sales","Transactions","Items"]

In [14]:
with open("oym_trans.txt", 'w') as outfile:
    outfile.write(",".join(headers) + "\n")
    for row in owner_year_month:
         outfile.write(",".join([str(item) for item in row])+ '\n')

In [15]:
input_file2 = "oym_trans.txt"

cur.execute('''DROP TABLE IF EXISTS Owner_Year_Month_Trans''') 
cur.execute('''CREATE TABLE Owner_Year_Month_Trans (
    Owner_ID INTEGER,
    Year TIMESTAMP,
    Month TIMESTAMP,
    Sales FLOAT,
    Transactions INTEGER,
    Items INTEGER)''')

with open(input_file2, 'r', encoding = 'Latin-1') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        cur.execute('''
        INSERT INTO Owner_Year_Month (Owner_ID, Year, Month, Sales, Transactions, Items)
        VALUES (?,?,?,?,?,?)''', line) #make sure that the ? match up with the column labels for value insertion

        db.commit()

## Task 3- Sales by Product Description by Year/Month

In [54]:
table3_salespdym = '''SELECT tr.department, dl.dept_name,
    (EXTRACT(year FROM datetime)) AS Year,
    (EXTRACT(month FROM datetime)) AS Month,
    upc,
    description,
    tr.department AS dept_no,
    SUM(total) as Sales,
    COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
    SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `my-project-366122.wedge.transArchive*` AS tr
    
    LEFT OUTER JOIN `my-project-366122.wedge.department_lookup` AS dl ON tr.department = dl.department
    
    WHERE card_no != 3
    AND tr.department != 0
    AND tr.department != 15
    AND trans_status != 'M'
    AND trans_status != 'C'
    AND trans_status != 'J'
    AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
    GROUP BY Year, Month, upc, description, dept_no, dl.dept_name
    ORDER BY description, Year, Month DESC'''    
    
prod_descript = client.query(table3_salespdym,location="US")
headers = ["department","dept_name","Year","Month","upc","description","dept_no","Sales","Transactions","Items"]

In [55]:
with open("sales_by_product_description.txt", 'w') as outfile:
    outfile.write(",".join(headers) + "\n")
    for row in prod_descript:
         outfile.write(",".join([str(item) for item in row])+ '\n')

In [57]:
input_file3 = "sales_by_product_description.txt"

cur.execute('''DROP TABLE IF EXISTS Item_Description_by_Month''') 
cur.execute('''CREATE TABLE Item_Description_by_Month (
    department FLOAT,
    dept_name TEXT,
    Year TIMESTAMP,
    Month TIMESTAMP,
    upc STRING,
    description STRING,
    dept_no INTEGER,
    Sales FLOAT,
    Transactions INTEGER,
    Items INTEGER)''')

with open(input_file3, 'r', encoding = 'Latin-1') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        cur.execute('''
        INSERT INTO Item_Description_by_Month (department, dept_name, Year, Month, upc, description, dept_no, Sales, Transactions, Items)
        VALUES (?,?,?,?,?,?,?,?,?,?)''', line)

        db.commit()
